<h1 align="center" style="margin-bottom: 20px;">Business Cases with Data Science 2024-25</h1>
<h3 align="center" style="margin-top: 20px; margin-bottom: 20px;">Case 4: AI - Powered Chatbot</h3>
<h5 align="center" style="margin-top: 20px; margin-bottom: 0px;">Notebook 1 

### Group B - Members:
- Ana Marta Azinheira | 20240496@novaims.unl.pt
- Bráulio Damba | 20240007@novaims.unl.pt
- Jan-Louis Schneider | 20240506@novaims.unl.pt
- Sofia Jacinto | 20240598@novaims.unl.pt

# Notebook Objective

- This notebook is designed to test our AI-Powered Chatbot. Throughout the notebook, we develop and refine various prompts to interact with our model. To support this notebook, we include a separate .py file containing the core functions used by the chatbot. At the end of the notebook, users will be able to test the chatbot in a simulated environment.

# Importing Libraries and Others

In [3]:
# Imports
import os
import json
import time
import pandas as pd
from openai import AzureOpenAI
from PIL import Image
from IPython.display import Markdown, display
import pickle
from datetime import datetime, timedelta
import fitz
from docx import Document  

#pip install pymupdf python-docx
#!pip install python-docx
#!pip install streamlit

In [4]:
# Import methods from utils.py
from utils import (
    create_assistant,
    create_thread,
    check_assistant_exists,
    load_and_upload_files,
    add_message_to_thread,
    display_messages,
    send_message_to_assistant
)

In [ ]:
# Set API key and endpoint
api_key = 'yourAPIKey'
endpoint = 'https://ai-bcds.openai.azure.com/'

In [6]:
# Constants
current_folder = os.getcwd()
data_folder = current_folder
data_folder_full_path = os.path.abspath(data_folder)

assistantFilename = 'AssistantID.TXT'
assistant_id = None
assistant = None
vector_data = 'vector_store.pkl'   # Where uploaded data will be/is saved

displayedMessagesIDs = []

In [7]:
# Initialize the Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint = endpoint,
    api_key= api_key,
    api_version="2024-05-01-preview")

In [8]:
# Load the link_map created, that links every document with a source link
with open("document_links.json", "r", encoding="utf-8") as f:
    doc_link_map = json.load(f)

In [9]:
# Load or upload documents 
if os.path.exists(vector_data):
    with open(vector_data, "rb") as file:
        vector_store = pickle.load(file)
    print("Loaded existing vector store from file.")
else:
    print("Uploading documents and creating vector store...")
    vector_store = load_and_upload_files(client, link_map=doc_link_map)
    print("Documents uploaded.")

Loaded existing vector store from file.


# Rules/Prompts for the Model

In [10]:
# Role of the model
aRole = (
    "És um assistente virtual da seguradora Fidelidade, fiável e rápido, que apoia os colaboradores durante os atendimentos a clientes.\n"
    "O teu objetivo é fornecer respostas claras, corretas e rápidas, ajudando os colaboradores da Fidelidade a responder com confiança.\n"
    "Entendes o contexto da conversa e considera sempre as perguntas anteriores para manter coerência nas respostas.\n"
    "Se o colaborador fizer uma pergunta de seguimento, lembre-se do que foi dito antes.\n"
    "Não interages diretamente com o cliente final, mas atua como um suporte eficiente para os colaboradores.\n"
    "Responde de forma natural, amigável e clara."
)

In [11]:
with open("prompt_rules.txt", "r", encoding="utf-8") as f:
    prompt_rules = f.read()

In [12]:
# Load or create assistant using existing vector store

# Try load existing assistant
if os.path.exists(assistantFilename):
    with open(assistantFilename, "r") as file:
        assistant_id = file.read().strip()

    # Check if assistant exists in azure
    exists, assistant = check_assistant_exists(client, assistant_id)
    if exists:
        # Load assistant with new role
        assistant = client.beta.assistants.update(
            assistant_id=assistant_id,
            instructions=aRole, 
            tool_resources={    # The documents made available for the model
                "file_search": {
                    "vector_store_ids": [vector_store.id]
                }
            }
        )
        print("Using existing assistant:", assistant_id)

    else:  # If assistant exist but not valid in azure
        print("Assistant ID found, but not valid in API. Creating new one...")
        assistant_id = None   # Marker to create new assistant 

else:   # If no assistant found
    print("ℹNo assistant ID file found. Creating new assistant...")
    assistant_id = None   # Marker to create new assistant

# No valid assistant --> create new one
if assistant_id is None:
    print("Creating new assistant...")
    assistant = create_assistant(client, aRole, assistantFilename)
    assistant_id = assistant.id

    assistant = client.beta.assistants.update(
        assistant_id=assistant.id,
        tool_resources={
            "file_search": {
                "vector_store_ids": [vector_store.id]
            }
        }
    )
    print("New assistant created and linked.")

Using existing assistant: asst_afxDUFnwot8aRxGIbDcmPyiv


In [13]:
# New thread
thread = create_thread(client)

# Testing the Chat Bot

In [ ]:
# WRITE 'Obrigado, até à próxima!' TO TERMINATE CHAT or WRITE 'Nova Conversa' to reset chat

while True:
    user_input = input("You: ")
    full_prompt = f"{prompt_rules}\n\nUser question: {user_input}"

    if user_input.strip().upper() == "Obrigado, até à próxima!":
        print("Obrigado pelo seu contacto. Sempre que precisar estarei aqui. Para que a vida não pare.")
        break
    elif user_input.strip().upper() == "Nova Conversa":
        print("Starting a new chat...")
        thread = create_thread(client)
    else:
        send_message_to_assistant(client, thread, assistant, user_input, full_prompt, displayedMessagesIDs)

You: bom dia!
Thinking...


Assistant: Bom dia! Como posso ajudá-lo hoje? Se tiver alguma pergunta específica, estou à disposição.

You: fala-me de seguros
Thinking...


Assistant: Claro! Aqui estão alguns conceitos básicos sobre seguros:

- **Definição de Seguro**: É um contrato que oferece proteção financeira contra riscos, como danos, perdas ou responsabilidades, em troca do pagamento de um prêmio.

- **Tipos de Seguros**:
  - **Seguro de Vida**: Proporciona suporte financeiro aos beneficiários em caso de falecimento do segurado.
  - **Seguro de Saúde**: Cobre despesas médicas e hospitalares para o segurado.
  - **Seguro Automóvel**: Protege contra danos ao veículo e lesões corporais resultantes de acidentes de carro.
  - **Seguro Residencial**: Cobre danos à propriedade e bens dentro da casa, incluindo riscos como incêndio e roubo.
  - **Seguro de Viagem**: Oferece cobertura para imprevistos que possam ocorrer durante viagens, como cancelamentos e emergências médicas.

- **Princípios de Funcionamento**:
  - **Prêmio**: Valor pago pelo segurado para manter a cobertura do seguro.
  - **Cobertura**: Limite de proteção disponível em caso de sinistro.
  - **Franquia**: Valor que o segurado deve pagar antes que o seguro entre em efeito para o pagamento de um sinistro.

- **Benefícios**:
  - Proporciona segurança financeira
  - Ajuda a gerenciar riscos
  - Promove a tranquilidade sabendo que há proteção em caso de imprevistos

Se precisar de mais detalhes específicos sobre algum tipo de seguro ou produtos da Fidelidade, estou à disposição para ajudar!

You: e podes aconselhar qual o melhor?
Thinking...


Assistant: Para responder à sua pergunta de forma precisa, preciso de mais contexto sobre qual tipo de seguro ou produto você está se referindo. Cada tipo de seguro ou produto da Fidelidade pode ter características diferentes e ser mais adequado para diferentes necessidades. 

Seria útil especificar se está interessado em:

- Seguro de vida
- Seguro saúde
- Seguro automóvel
- Outro tipo de seguro ou produto financeiro

Dessa forma, poderei oferecer informações mais relevantes e adequadas.